In [1]:
# This should be able to take TA indicators and feed them to Tensorflow

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd

import ag.bittensor.utils.options as options
import ag.bittensor.ai.make_data as make_data
import ag.bittensor.utils.talib as talib

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical
tflearn.init_graph(num_cores=8, gpu_memory_fraction=0.5)
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

c:\users\kinet\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Globals
datasmith = make_data.MakeData()
TA = talib.TALib()
timeframe = '5T'
period = 12
# get sample
sample = datasmith.random_filename

In [4]:
def unzero(df):
    df = df.replace(0,'NaN')
    df = df.dropna(how='all',axis=0)
    df = df.replace('NaN', 0)
    df.len = len(df)
    return df

In [5]:
def make_sample(sample, period, timeframe):
    def unzero(df):
        df = df.replace(0,'NaN')
        df = df.dropna(how='all',axis=0)
        df = df.replace('NaN', 0)
        df.len = len(df)
        return df
    # make data
    datasmith.dataframe = sample
    datasmith.candles = timeframe
    df = datasmith.candles
    # get all the TA
    B1, MA, B2 = TA.BBANDS(df, period)
    EMA = TA.EMA(df, period+1)
    RSI = unzero(TA.RSI(df, period))
    MOM = unzero(TA.MOM(df, period-3))
    CCI = TA.CCI(df, period)
    # concatenate with RSI first because it has no x axis
    input_df = pd.DataFrame({
        'RSI': RSI.values,
    }, index=MOM.index[:len(RSI)])
    for i in [df['Close'],B1,MA,B2,EMA,MOM,CCI]:
        input_df = input_df.join(i)
    # fill gaps
    input_df_ = input_df.fillna(method='bfill')
    
    # make NP array
    input_df = np.array(input_df_)
    print('Starting Shape',input_df.shape)
    
    # normalize
    input_df = np.diff(input_df, axis=0) / input_df[1:] * 100
    print('Shape after np.diff: {}'.format(input_df.shape))
    
    # pull infinity
    for i in range(len(input_df)):
        for k, j in enumerate(input_df[i]):
            if not np.isfinite(j):
                input_df[i][k] = 0
        # break
    
    # scale it from -1 to 1 
    # this is binary... i dont know if i like it...
    scaler = MaxAbsScaler()
    input_df = scaler.fit_transform(input_df)
    print('Shape after scaler tranformation:', input_df.shape)
    
    # make polynomially complex( add complexity for no good reason )
    # poly = PolynomialFeatures(degree=4, interaction_only=True)
    # input_df = poly.fit_transform(input_df)
    # print('Shape after Poly tranformation:', input_df.shape)
    
    
    # return the complete dataset for index reference
    # AND an iterator so that you know when your done
    return input_df_, input_df, iter(input_df)
df, numdf, samples = make_sample(sample, period, timeframe)

Starting Shape (35889, 8)
Shape after np.diff: (35888, 8)


c:\users\kinet\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in true_divide
c:\users\kinet\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


Shape after scaler tranformation: (35888, 8)


In [10]:
def get_reward(action, position, df):
    change = unzero(df['Close'].pct_change(5).diff().cumsum().fillna(method='bfill'))
    trend = change.values[position]
    buy_hold = change.values[-1]
    right_answer = 2  # do nothing... is usually the right answer
    if (trend*100) > 1:
        right_answer = 0  # buy if going up
    elif (trend*100) < -1:
        right_answer = 1  # sell if going down

    if action == right_answer:
        reward = 1
    else: reward = -1
    return reward, buy_hold
# reward, bh = get_reward(0, 100, df)

In [8]:
# print(numdf)

In [17]:
samples = iter(numdf)
print(numdf.shape)
for i, sample_ in enumerate(samples):
    sample_ = sample_.reshape(-1,1)
    print('sample num:', i)
    print('1 sample shape', sample_.shape)
    print('Sample:')
    print(sample_)
    print('Action:', 1)
    
    reward, _ = get_reward(1, i, df)
    print('Reward:', reward)
    if i == 12:
        break

(35888, 8)
sample num: 0
1 sample shape (8, 1)
Sample:
[[-0.00036856]
 [-0.00401481]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [-0.00011223]
 [ 0.        ]]
Action: 1
Reward: -1
sample num: 1
1 sample shape (8, 1)
Sample:
[[-1.97526548e-18]
 [ 4.01359405e-03]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-4.67836257e-02]
 [ 0.00000000e+00]]
Action: 1
Reward: -1
sample num: 2
1 sample shape (8, 1)
Sample:
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-6.67804748e-02]
 [ 3.13136643e-01]
 [ 2.31830507e-01]
 [ 3.17582411e-02]
 [-5.92592593e-02]
 [-1.79565303e-11]]
Action: 1
Reward: -1
sample num: 3
1 sample shape (8, 1)
Sample:
[[ 7.33240719e-03]
 [ 0.00000000e+00]
 [-2.39914346e-01]
 [ 3.12342887e-01]
 [ 4.56061338e-01]
 [ 2.59282937e-02]
 [ 0.00000000e+00]
 [-8.90903290e-12]]
Action: 1
Reward: -1
sample num: 4
1 sample shape (8, 1)
Sample:
[[-1.20633284e-02]
 [ 0.00000000e+00]
 [-4.31117777e-02]
 [ 5.93165804e-02]
 [ 8.30079075e-02]
 [ 2.13360